In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [ ]:
# Load the datasets
game_data = pd.read_csv('data/game_info.csv', index_col=0)
user_data_train = pd.read_csv('data/user_data_train_no_comments.csv', index_col=0)
user_data_validation = pd.read_csv('data/user_data_validation_no_comments.csv', index_col=0)
user_data_train_validation = pd.concat([user_data_train, user_data_validation])
user_data_test = pd.read_csv('data/user_data_test_no_comments.csv', index_col=0)

In [ ]:
# categories of game data to consider in content-based filtering for computing similarities
content_categories = ['Title', 'Publisher', 'Year', 'Genre', 'Platform']

def get_bag(category):
    '''
    Return a bag of words for tf-idf vectorization given the category name
    '''
    processing_func = None
    if category == 'Title':
        processing_func = lambda s: s.split()
    elif category == 'Publisher':
        processing_func = lambda s: s.split(',')
    elif category == 'Genre':
        processing_func = lambda s: s.split(';')
    else:
        processing_func = lambda s: [str(s)]

    return game_data[category].apply(processing_func)

def get_similarity_matrix(categories, vectorizer):
    '''
    Return the cosine similarity matrix using the given categories
    '''
    assert categories # assert it is not an empty list

    # retrieve a bag of words per game to vectorize
    bag_of_texts = get_bag(categories[0])
    for category in categories[1:]:
        bag_of_texts += get_bag(category)

    # create a tf-idf vectorizor that filters out non-alphanumeric characters and ignores case
    tf = TfidfVectorizer(analyzer=lambda i: map(lambda s: ''.join(filter(str.isalnum, s.lower())), i))

    # compute the tf-idf matrix
    tfidf_matrix = tf.fit_transform(bag_of_texts)

    # compute and return the cosine similarity matrix
    return cosine_similarity(tfidf_matrix)

# compute the cosine similarity matrix
similarity_matrix = get_similarity_matrix(content_categories)

In [ ]:
def content_based_prediction(test_dataset, similarity_matrix, rating):
    '''
    Make a prediction about the userscore of a particular rating based on the user's rating of other games in the test_dataset
    '''
    # get all ratings from the user
    user_ratings = test_dataset[test_dataset.Username == rating.Username]
    user_ratings = user_ratings.set_index('Game_ID')
    user_ratings = user_ratings.drop(rating.Game_ID, errors='ignore') # Remove the game rating we are predicting
    game_sims = similarity_matrix[user_ratings.index, rating.Game_ID]
    sim_sum = sum(game_sims)
    if sim_sum == 0:
        return np.NaN
    else:
        return sum(user_ratings.Userscore * game_sims) / sim_sum

In [ ]:
def content_based_predictions(test_dataset, similarity_matrix):
    '''
    Make predictions for every rating from a given test_dataset using a given similarity matrix.
    If a prediction can not be made, the game's mean userscore from game_mean_userscores is predicted instead.
    If a prediciton can not be made and the game is not in the game_mean_userscores dataset, predict the fallback_score.
    '''
    return test_dataset.apply(lambda row: content_based_prediction(test_dataset, similarity_matrix, row), axis=1)

def game_mean_userscores_predictions(test_dataset, training_dataset):
    '''
    For each row in the test_dataset, give the game's average userscore if available.
    '''
    game_mean_userscores = training_dataset.groupby('Game_ID').mean()
    return game_mean_userscores.reindex(test_dataset.Game_ID).set_index(test_dataset.index).Userscore

def training_mean_predictions(test_dataset, training_dataset):
    '''
    For each row in the test_dataset, give the game's average userscore if available.
    '''
    return pd.Series(np.ones(len(test_dataset.Userscore)) * training_dataset.Userscore.mean(), index=test_dataset.index)

def compute_rmse(test_dataset, predictions):
    '''
    Compute RMSE on the test_dataset given the predictions.
    If a prediction is NaN, then the prediction is ignored and does not contribute to the RMSE.
    The index of the predictions must be a subset or equal of the test_dataset index.
    '''
    num_dropped = len(test_dataset) - len(predictions[predictions.notna()])
    if num_dropped:
        print('NaN predictions detected.', num_dropped, ' ( out of', len(test_dataset), ') data points have been excluded from the RMSE.')
    return mean_squared_error(predictions[predictions.notna()], test_dataset.Userscore[predictions.notna()], squared=False)


In [ ]:
training_dataset = user_data_train_validation
test_dataset = user_data_test

In [ ]:
'''
Make predictions using the content-based filtering
'''
preds = content_based_predictions(test_dataset, similarity_matrix)
preds_rmse = compute_rmse(test_dataset, preds)
preds_rmse

In [ ]:
'''
Baseline 1 - Predict only the mean user score from training data
'''
tm_preds = training_mean_predictions(test_dataset, training_dataset)
tm_preds_rmse = compute_rmse(test_dataset, tm_preds)
tm_preds_rmse

In [ ]:
'''
Baseline 2 - Predict the mean userscore of a game from the training data
'''
gmu_preds = game_mean_userscores_predictions(test_dataset, training_dataset)
gmu_preds_rmse = compute_rmse(test_dataset, gmu_preds)
gmu_preds_rmse

In [ ]:
'''
Finished with with the datasets
'''
del user_data_train
del user_data_validation
del user_data_train_validation
del user_data_test

In [ ]:
'''
Permutation test
'''
# Set a seed for reproducible results
seed = 8175615
np.random.seed(seed)

# Set number of permutations
num_permutations = 100
RMSEs = np.zeros(num_permutations)

# Create new data splits

user_data = pd.read_csv('data/user_data_no_comments.csv', index_col=0)
# Remove anonymous users
user_data = user_data[user_data["Username"] != "[Anonymous]"]
user_data = user_data[user_data["Username"] != "AnonymousMC"]
# Remove users with less than 3 reviews
user_data = user_data.groupby('Username')
user_data = user_data.filter(lambda x: len(x) > 2)

for i in range(num_permutations):

    print('iteration', i)

    # Shuffle the user score column of the user data
    permuted_user_data = user_data.copy()
    permuted_userscores = user_data.Userscore.sample(frac=1)
    permuted_userscores.index = user_data.Userscore.index
    permuted_user_data.Userscore = permuted_userscores
    user_groups = permuted_user_data.groupby('Username')

    # Take 20% of the data to test on
    _, test_groups = train_test_split(
        list(user_groups), 
        train_size=.8
    )
    randomized_test_set = pd.concat([x[1] for x in test_groups])

    # Make predictions
    preds = content_based_predictions(randomized_test_set, similarity_matrix)
    rmse = compute_rmse(randomized_test_set, preds)
    RMSEs[i] = rmse
    print('RMSE', rmse)

In [ ]:
'''
Save RMSEs to a CSV
'''
RMSE_series = pd.Series(RMSEs)
RMSE_series.to_csv('permutation_test_scores.csv')

In [ ]:
'''
Show histogram of permutation test results alongside the true test result
'''
plt.hist(RMSEs)
plt.axvline(preds_rmse, color='k', linestyle='dashed', linewidth=1)
plt.xlabel('RMSE')
plt.ylabel('Count')
plt.title('Histogram of Permutation Test Scores')
plt.savefig('perm_test_hist.png', dpi=600)